# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [ ]:
%pip install ucimlrepo

In [22]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [24]:
from sklearn.model_selection import train_test_split

#Create feature dataframe X and target dataframe Y
X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

#split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Print the shapes of the resulting datasets to verify
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random state in the train_test_split function is an integer value, in this case, 42, that acts as a "seed" for the random number generator. This seed is used to shuffle and split the data consistently each time the code is run.

Why is it Useful?
Setting a random state has two main benefits:

Reproducibility: When a random state is set, the split between training and testing data remains the same every time the code is executed. This allows for reproducible results, making it easier to validate and compare model performance across different experiments or by different researchers. 

Consistency in Model Evaluation: By keeping the data split consistent, we ensure that any performance improvements (or declines) are due to model adjustments rather than changes in the data. This is essential in machine learning, where model performance can be affected by even small variations in the data.

In this assignment, using random_state=42 in train_test_split guarantees that anyone using this code will work with the same 70-30 data split, which is particularly useful for collaborative projects

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import set_config




#Let's identify numerical and categorical columns
numerical_columns = X.select_dtypes(include=['int64', 'float']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

#create the numerical pipeline
numerical_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())    
])

#create the categorical pipeline
categorical_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='most_frequent')),
  ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))  
])

#create the columnTransformer
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_columns),
    ('cat', categorical_pipeline, categorical_columns)
])

#Fit the preprocessor to the training data
X_train_preprocessed = preprocessor.fit_transform(X_train)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


# Define the categorical and numerical columns
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 
                    'relationship', 'race', 'sex', 'native-country']
numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 
                  'capital-loss', 'hours-per-week']

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Scale numerical features
        ('cat', OneHotEncoder(), categorical_cols)   # One-hot encode categorical features
    ])


# Create the pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Preprocessing step
    ('classifier', RandomForestClassifier(random_state=42))  # Classifier step
])

# Display the pipeline to confirm structure
model_pipeline


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat', OneHotEncoder(),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
import pandas as pd
import numpy as np

#1. Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
    
}

#Cross-validation with cross_validate()
cv_results = cross_validate(
    model_pipeline,
    X_train,
    Y_train,
    cv=5,
    scoring=scoring,
    return_train_score=True
)



Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [28]:
#2. Display fold-level results as a sorted Dataframe 
cv_results_df = pd.DataFrame(cv_results)
cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)
print("Cross-validation results sorted by test negative log loss:")
print(cv_results_df)

Cross-validation results sorted by test negative log loss:
    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
1  21.997874    0.355751          -0.369554           -0.081352      0.901962   
2  21.544134    0.356589          -0.376757           -0.081473      0.901477   
3  22.050532    0.441285          -0.379715           -0.082503      0.905383   
4  22.114877    0.344524          -0.388778           -0.081847      0.901937   
0  21.740998    0.110586                NaN           -0.081462           NaN   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
1            1.0       0.844483             1.0                0.765513   
2            1.0       0.853006             1.0                0.774985   
3            1.0       0.860246             1.0                0.782533   
4            1.0       0.854322             1.0                0.775239   
0            1.0            NaN             1.0                     NaN   

   

Calculate the mean of each metric. 

In [34]:
# 3. Calculate the mean of each metric
mean_results = cv_results_df.mean()
print("Mean Results:")
print(mean_results)


Mean Results:
fit_time                   21.889683
score_time                  0.321747
test_neg_log_loss          -0.378701
train_neg_log_loss         -0.081727
test_roc_auc                0.902690
train_roc_auc               1.000000
test_accuracy               0.853014
train_accuracy              1.000000
test_balanced_accuracy      0.774567
train_balanced_accuracy     1.000000
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [35]:
#Calculate performance metrics on the test set 
model_pipeline.fit(X_train, Y_train)
Y_pred_proba = model_pipeline.predict_proba(X_test) # use probabilities for Log loss and ROC AUC
Y_pred = model_pipeline.predict(X_test) # Use Predictions for accuracy and balanced accuracy

test_metrics = {
    'test_neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'test_roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # Use probabilities for the positive class
    'test_accuracy': accuracy_score(Y_test, Y_pred),
    'test_balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

print("\nTest set performance metrics:")
print(test_metrics)



Test set performance metrics:
{'test_neg_log_loss': 0.382050638488725, 'test_roc_auc': 0.8997459548416978, 'test_accuracy': 0.8540280479066434, 'test_balanced_accuracy': 0.77412739183589}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

transforming income into a binary format immediately on load is a convenient, efficient, and error-preventing step that ensures the target variable is ready for analysis and modeling from the start.

Recoding the target variable immediately upon loading the data is convenient for several reasons:

1. Simplifies Target Representation: By converting income from a string (">50K" and "<=50K") to a binary format (0 or 1) right away, we make it easier to work with in machine learning models, which typically expect numerical target values. This binary format directly represents the classification labels (0 for <=50K and 1 for >50K), making it suitable for binary classification tasks.

2. Reduces Redundant Processing: Recoding the target variable at load time eliminates the need to perform this transformation multiple times throughout the data pipeline. It standardizes the income column format early in the workflow, so we don’t have to remember to transform it later, which can help prevent mistakes.

3. Enables Consistent Processing in Pipelines: By handling this transformation at the beginning, the pipeline can focus on data preprocessing and model fitting without additional steps to process the target variable. This also makes cross-validation and other evaluation methods easier to implement, as the target variable is already in the required format.

4. Minimizes Data Cleaning Steps: Converting income to a numerical variable directly in the loading step also strips any trailing whitespaces and standardizes it, which helps avoid potential issues with inconsistent string formats that could arise if the data were handled as-is.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.